## Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.8.0'

# Data Preprocessing

## Importing Dataset

In [ ]:
dataset = pd.read_csv('./data/Churn_Modelling.csv')

In [ ]:
X = dataset.iloc[ : , 3:-1].values    # storing indep data onm X
y = dataset.iloc[: , -1].values # storing dep data on y

# Note that first three cols are skipped as they are relevenat ot the data model

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


## Encoding Categorical Data

### Label Encoding of Gender Col

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[: , 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


### One Hot Encoding the Country Col

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder='passthrough')
# index passed is 1 as poisition of col changed when we skipped first three col while loading to var X

X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting Dataset into training set and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [ ]:
## Feature Scaling is fundamental and important for Deep Learning

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test= sc.fit_transform(X_test)

# Part 2 : Building ANN

## Initializng ANN

In [ ]:

# either sequential or computational graph

ann = tf.keras.models.Sequential()


## Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation= 'relu'))
# units denote the no of neurons for hidden layer
# relu corresponds to rectifier

## Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation= 'relu'))

## Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 1, activation= 'sigmoid'))
# units is 1 as we need only one output neuron as resultant value is boolean - either 0 or 1
# sigmoid function wil not only ptovide prediction, but the probability of outcome being 1

# softmax function should be used as activation function when no of categories to be predicted is more than 2


# Training the ANN

## Compiling the ANN with optimizer loss function and metric which is accuracy 

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics= ['accuracy'])

# loss functiuon should be 'binary_crossentropy' when we have to predict binary outcomes such as 0 or 1, yes or no
# for non binary categorical outcomes, 'categorical_crossentropy' should be used

## Training the ANN on training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

# batch_size is a hyperparameter and default value is 32

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5387 - accuracy: 0.7533
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4690 - accuracy: 0.7976
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4465 - accuracy: 0.8046
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4311 - accuracy: 0.8092
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4189 - accuracy: 0.8156
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4066 - accuracy: 0.8209
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3933 - accuracy: 0.8265
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3792 - accuracy: 0.8380
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3694 - accuracy: 0.8471
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3637 - accura

## Part 4 : Making the prediction and evaluating the model

## Predicting the result of single observation

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer ?

In [ ]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000 ]])) > 0.5)

# input for predict method is always a 2D array..hence doule sq brackets
# Anothre point is that as data was encoded for gender and country, the data passed for the same should 
# be encoded in similar format
# Also feature scaling is to be applied on the data and also transform method should be called. If fit transform method is called, then it
# consider the data as input to calcualte to find std value


[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

Important note 1: Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

Important note 2: Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

## Predicitng the test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1532   63]
 [ 209  196]]


0.864